### Multiple turtlebot simulator with rapyuta.io.

#### 1.IDs
you can copy from rapyuta.io console

In [1]:
AUTH_TOKEN = 'EfTVySQLTu57qrXgtr3NhBs2HpjjPCh3wwVZVYRD'
PROJECT_ID = 'project-fycsplhkhcymtvsmhjadjlae'
deployment_name_suffix = '_yu'

#gazebo
SIMULATOR_PACKAGE_ID = 'pkg-zbwnbmougwdbajbhmymsuxsj'
SIMULATOR_PLAN_ID = 'plan-raqssiekewbjcmsmktxuxtvd'
SIMULATOR_COMPONENT = 'gazebo'

#rosbridge
ROSBRIDGE_PACKAGE_ID = 'pkg-znheuolmabxoqwdzaafberee'
ROSBRIDGE_PLAN_ID = 'plan-shzbvioqwvrbwswpbtalppvp'

#ui
UI_PACKAGE_ID = 'pkg-uotxzftpaxxkcddilhpptozl'
UI_PLAN_ID = 'plan-suvouqjarpgmkuwllgmlqttr'

#app_multi_tv1_external
APP_PACKAGE_ID = 'pkg-cdgibmehhzlfvdktpgizgwos'
APP_PLAN_ID = 'plan-beqwcwoikdlbjnpsvfgmcxyl'
APP_COMPONENT = 'turtle'

#params
sim_params = {'WORLD':'turtlebot3_world', 'VNC_PASSWORD':'rapyuta'}
robot_params = [
    {'TURTLEBOT3_MODEL':'burger', 'ROBOT_NAME':'turtle0', 'X_POS':-0.5, 'Y_POS':-0.5, 'YAW':0.00},
    {'TURTLEBOT3_MODEL':'waffle', 'ROBOT_NAME':'turtle1', 'X_POS': 0.5, 'Y_POS': 0.5, 'YAW':3.14},
]
robot_num = len(robot_params)

#### 2. import module and create Client Instance

In [2]:
from rapyuta_io import Client
client = Client(AUTH_TOKEN, PROJECT_ID)

#### 3.Simulator deployment

In [3]:
simulator_package = client.get_package(SIMULATOR_PACKAGE_ID)
simulator_configuration = simulator_package.get_provision_configuration(SIMULATOR_PLAN_ID)
for key in sim_params:
    simulator_configuration.add_parameter(SIMULATOR_COMPONENT, key, sim_params[key])

simulator_deployment = simulator_package.provision(
                deployment_name = 'sim' + deployment_name_suffix,
				provision_configuration = simulator_configuration)
result = simulator_deployment.poll_deployment_till_ready(retry_count=50, sleep_interval=6)
print('gazebo started: ' + result['componentInfo'][0]['networkEndpoints']['vnc'])

gazebo started: https://inst-sjelpocyjzpbwqmfnohhlznh-jsdwuc.ep-r.io:443


#### 4.App deployments

In [4]:
app_deployments = [None, None]
app_package = client.get_package(APP_PACKAGE_ID)
for i in range(robot_num):
    name = robot_params[i]['ROBOT_NAME']
    app_configuration = app_package.get_provision_configuration(APP_PLAN_ID)
    app_configuration.add_dependent_deployment(simulator_deployment)
    for key in robot_params[i]:
        app_configuration.add_parameter(APP_COMPONENT, key, robot_params[i][key])
    app_configuration.set_component_alias(APP_COMPONENT, name)
    app_deployments[i] = app_package.provision(
                    deployment_name = name + deployment_name_suffix,
                    provision_configuration = app_configuration)
    
for i in range(robot_num):
    app_deployments[i].poll_deployment_till_ready(retry_count=50, sleep_interval=6)
    print(robot_params[i]['ROBOT_NAME'] + deployment_name_suffix + ' started')

turtle0_yu started
turtle1_yu started


#### 5.Rosbridge deployment

In [5]:
rosbridge_package = client.get_package(ROSBRIDGE_PACKAGE_ID)
rosbridge_configuration = rosbridge_package.get_provision_configuration(ROSBRIDGE_PLAN_ID)
# rosbridge_configuration.add_dependent_deployment(simulator_deployment)
for dep in app_deployments:
        rosbridge_configuration.add_dependent_deployment(dep)

        
rosbridge_deployment = rosbridge_package.provision(
                deployment_name = 'rosbridge' + deployment_name_suffix,
				provision_configuration = rosbridge_configuration)
rosbridge_deployment.poll_deployment_till_ready()
print('rosbridge started')

rosbridge started


#### 6.UI deployment

In [6]:
ui_package = client.get_package(UI_PACKAGE_ID)
ui_configuration = ui_package.get_provision_configuration(UI_PLAN_ID)
ui_configuration.add_dependent_deployment(rosbridge_deployment)
        
ui_deployment = ui_package.provision(
                deployment_name = 'ui' + deployment_name_suffix,
				provision_configuration = ui_configuration)
result = ui_deployment.poll_deployment_till_ready()
print('ui started ' + result['componentInfo'][0]['networkEndpoints']['UserInterface'])

ui startedhttps://inst-cshqpapjwqfheluphaeyrcmz-zjoach.ep-r.io:443


#### 7.Deplovision all deploment

In [7]:
retry_limit = 3
simulator_deployment.deprovision(retry_limit)
for i in range(robot_num):
    app_deployments[i].deprovision(retry_limit)
rosbridge_deployment.deprovision(retry_limit)
ui_deployment.deprovision(retry_limit)

True